### **Operations on New Virtual Machines**

In [0]:
!pip install tensorflow-gpu

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd "drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


In [0]:
!ls
!nvidia-smi

architecture.py       tiny_yolo_rbc_4.h5
BoundBox.py	      utils.py
Football	      vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5
inception_backend.h5  yolo-keras-ball-model.h5
Keras-Yolo.ipynb      yolo-keras-ball-weights.h5
Keras-Yolo-TPU.ipynb  yolo-keras-model.h5
main.ipynb	      yolo-keras-raccoon-model.h5
mobilenet_backend.h5  yolo-keras-raccoon-weights.h5
__pycache__	      yolo-keras-rbc-model.h5
Raccoon		      yolo-keras-weights.h5
RBC		      Yolo-predict.ipynb
tiny_yolo_backend.h5
Sun Jul 28 09:07:35 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+==============

### **Program Codes Start Here**

In [0]:
# from architecture import VGG16Net
from architecture import TinyYoloNet

Using TensorFlow backend.


In [0]:
anchors = [0.57273, 0.677385, 1.87446, 2.06253, 3.33843, 5.47434, 7.88282, 3.52778, 9.77052, 9.16828]
base_net = TinyYoloNet(net_input_size=224, 
                       anchors=anchors, 
                       n_class=1, 
                       weights_dir='tiny_yolo_backend.h5', 
                       labels=['raccoon'])

# base_net = VGG16Net(net_input_size=224, 
#                        anchors=anchors, 
#                        n_class=1, 
#                        weights_dir='vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5', 
#                        labels=['raccoon'])



feature_extractor = base_net.create_base_network(transfer_learning=False)

W0728 13:09:16.142059 140092248201088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0728 13:09:16.170955 140092248201088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0728 13:09:16.174596 140092248201088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0728 13:09:16.203291 140092248201088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0728 13:09:16.204527 1400922482

In [0]:
feature_extractor.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
model = base_net.create_model(feature_extractor)

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
model_1 (Model)              (None, 7, 7, 512)         14714688  
_________________________________________________________________
DetectionLayer (Conv2D)      (None, 7, 7, 30)          15390     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 5, 6)        0         
Total params: 14,730,078
Trainable params: 15,390
Non-trainable params: 14,714,688
_________________________________________________________________


### **Load Dataset**

In [0]:
from utils import parse_annotation, generate_Xy

In [0]:
!ls annotations/

In [0]:
imgs, annot = parse_annotation('Raccoon/annotations/', 'Raccoon/images/', ['raccoon'])
test_imgs, test_annot = parse_annotation('Raccoon/testannotations/', 'Raccoon/testimages/', ['raccoon'])

In [0]:
test_imgs[0]

In [0]:
X_train, y_train = generate_Xy(imgs, ['raccoon'], anchors, 7, 224, 1, base_net.normalize)
X_test, y_test = generate_Xy(test_imgs, ['raccoon'], anchors, 7, 224, 1, base_net.normalize)

In [0]:
print(X_train.shape)
print(y_train.shape)

(180, 224, 224, 3)
(180, 7, 7, 5, 6)


### **Train Model**

In [0]:
from keras.optimizers import Adam
learning_rate = 1e-4
optimizer = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model = base_net.compile_model(model, optimizer)

W0728 13:09:43.618672 140092248201088 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0728 13:09:43.635631 140092248201088 deprecation.py:323] From /content/drive/My Drive/Colab Notebooks/architecture.py:79: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.


In [0]:
model.fit(x=X_train, 
          y=y_train, 
          validation_data=(X_test, y_test), 
          epochs=25, 
          batch_size=16, 
          verbose=1)

Train on 180 samples, validate on 20 samples
Epoch 1/25
180/180 [==============================] - 1s 6ms/step - loss: 0.0033 - val_loss: 1.5082
Epoch 2/25
180/180 [==============================] - 1s 5ms/step - loss: 0.0031 - val_loss: 1.5762
Epoch 3/25
180/180 [==============================] - 1s 5ms/step - loss: 0.0028 - val_loss: 1.5416
Epoch 4/25
180/180 [==============================] - 1s 5ms/step - loss: 0.0030 - val_loss: 1.5339
Epoch 5/25
180/180 [==============================] - 1s 5ms/step - loss: 0.0028 - val_loss: 1.5184
Epoch 6/25
180/180 [==============================] - 1s 5ms/step - loss: 0.0027 - val_loss: 1.5502
Epoch 7/25
180/180 [==============================] - 1s 5ms/step - loss: 0.0029 - val_loss: 1.5844
Epoch 8/25
180/180 [==============================] - 1s 5ms/step - loss: 0.0035 - val_loss: 1.4939
Epoch 9/25
180/180 [==============================] - 1s 5ms/step - loss: 0.0036 - val_loss: 1.3730
Epoch 10/25
180/180 [==============================] - 

### **Evaluate Model**

In [0]:
base_net.evaluate(model, test_imgs)

/content/drive/My Drive/Colab Notebooks/utils.py:25: RuntimeWarning: overflow encountered in exp
  return 1. / (1. + np.exp(-x))


raccoon 0.8095
mAP: 0.8095


In [0]:
model.save('weights_models/yolov2-keras-raccoon-model.h5')
model.save_weights('weights_models/yolov2-keras-raccoon-weights.h5')